<a href="https://colab.research.google.com/github/mpily/FPL_AI_PROJECT/blob/main/UsingCleanedDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Sun* AI Project/

/content/drive/MyDrive/Sun* AI Project


In [5]:
%ls


cleaned_dataset/  data_fpl.csv  data_fpl.gsheet  data_fpl.xlsx


In [23]:
import pandas as pd
import numpy as np

In [7]:
train_mid = pd.read_csv("cleaned_dataset/train_MID.csv", index_col=0)
test_mid = pd.read_csv("cleaned_dataset/test_MID.csv", index_col=0)
train_copy = train_mid.copy()
test_copy = test_mid.copy()

train_mid["index"] = train_mid["name"] + train_mid["kickoff_time"].astype("str")
train_mid.drop_duplicates("index", keep="last", inplace=True)
train_mid = train_mid.set_index("index")
train_mid.drop(["kickoff_time"], axis=1, inplace=True)

test_mid["index"] = test_mid["name"] + test_mid["kickoff_time"].astype("str")

test_mid = test_mid.set_index("index")
test_mid.drop(["kickoff_time"], axis=1, inplace=True)

target = train_mid[["minutes", "GW"]]
train_mid.drop(["total_points", "minutes"], axis=1, inplace=True)
test_mid.drop(["total_points", "minutes"], axis=1, inplace=True)


for col in train_mid.columns:
    if train_mid[col].dtype == "object":
        if col not in ["team_x", "name"]:
            train_mid[col] = pd.factorize(train_mid[col])[0]
            test_mid[col] = pd.factorize(test_mid[col])[0]

train_mid["was_home"] = train_mid["was_home"].replace({True: 0, False: 1})
test_mid["was_home"] = test_mid["was_home"].replace({True: 0, False: 1})

test_mid = test_mid[train_mid.columns]

In [10]:
from sklearn.model_selection import train_test_split
leak_columns=["name", "team_x"]#columns that shouldnt be used in training fir fear of data leakage


In [11]:
x, val, y, y_val = train_test_split(
    train_mid.drop(leak_columns, axis=1), target["GW"], test_size=0.1, random_state=0
)

y = target["minutes"].loc[y.index]
y_val = target["minutes"].loc[y_val.index]


In [19]:
#STEP 1: USE RANDOM FOREST CLASSIFIER TO PREDICT WHETHER PLAYER WILL PLAY OR NOT
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
random_forest = Pipeline(
    [
        ("imp", SimpleImputer()),
        ("scaler", StandardScaler()),
        (
            "model",
            RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=1),
        ),
    ]
)

random_forest.fit(x,y)
print(confusion_matrix(random_forest.predict(val), y_val))

ImportError: ignored

In [17]:
print(random_forest.score(val,y_val))

0.8229278794402584


In [21]:
#Step 2: predicting points of starting players
train_mid = train_copy[train_copy["minutes"] > 0]
test_mid = test_copy[test_copy["minutes"] > 0]


# predict points
train_mid["index"] = train_mid["name"] + train_mid["kickoff_time"].astype("str")
train_mid.drop_duplicates("index", keep="last", inplace=True)
train_mid = train_mid.set_index("index")
train_mid.drop(["kickoff_time"], axis=1, inplace=True)

test_mid["index"] = test_mid["name"] + test_mid["kickoff_time"].astype("str")
test_mid = test_mid.set_index("index")
test_mid.drop(["kickoff_time"], axis=1, inplace=True)

target = train_mid[["total_points", "GW"]]
train_mid.drop(["total_points", "minutes"], axis=1, inplace=True)
test_mid.drop(["total_points", "minutes"], axis=1, inplace=True)

for col in train_mid.columns:
    if train_mid[col].dtype == "object":
        if col not in ["team_x", "name"]:
            train_mid[col] = pd.factorize(train_mid[col])[0]
            test_mid[col] = pd.factorize(test_mid[col])[0]

train_mid["was_home"] = train_mid["was_home"].replace({True: 0, False: 1})

test_mid["was_home"] = test_mid["was_home"].replace({True: 0, False: 1})

test_mid = test_mid[train_mid.columns]

x, val, y, y_val = train_test_split(
    train_mid.drop(leak_columns, axis=1), target["GW"], test_size=0.1, random_state=0
)


<ipython-input-21-c52b53b3c62d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_mid["index"] = train_mid["name"] + train_mid["kickoff_time"].astype("str")
/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
model = Pipeline(
    [
        ("imp", SimpleImputer()),
        ("scaler", StandardScaler()),
        (
            "model",
            RandomForestRegressor(random_state=0, max_depth=8, n_estimators=1000),
        ),
    ]
)

In [24]:
y = target["total_points"].loc[y.index]
y_val = target["total_points"].loc[y_val.index]

model.fit(x, y)
print(mean_absolute_error(model.predict(val), y_val))
print(
    f"Root Mean Squared Error {np.sqrt(mean_squared_error(model.predict(val), y_val))}"
)

1.78188135881104
Root Mean Squared Error 2.700333827684707


In [25]:
print(model.score(val,y_val))

0.12900481251957174
